In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 1
%aimport strife
import strife
import numpy as np
import numpy
import h5py
import bz2
import json

In [2]:
from IPython.parallel import Client

In [3]:
rc = Client()

In [4]:
dv = rc[:]

In [5]:
dv.use_dill()

<AsyncResult: use_dill>

In [6]:
%pxconfig -a --verbose

In [7]:
with dv.sync_imports():
    import strife
    import numpy
    import h5py
    import bz2
    import json

importing strife on engine(s)
importing numpy on engine(s)
importing h5py on engine(s)
importing bz2 on engine(s)
importing json on engine(s)


In [8]:
%%px --local
sample_frequencies = 'frequencies'
sample_snapshots = 'snapshots'

Async parallel execution on engine(s): all


<AsyncResult: finished>

In [9]:
%%px --local
params_index_to_value = dict(
    D=['0', '0.2', '0.4'],
    pgcost=[10, 30],
    seed=[1, 2],
)

Async parallel execution on engine(s): all


<AsyncResult: finished>

In [10]:
make_params_index_dict_list = lambda: [
    dict(
        seed=seed,
        D=D,
        pgcost=pgcost,
        cth=cth,
        sth=sth,
    )
    for D in range(3)
    for seed in range(2)
    for pgcost in range(2)
    for cth in range(10)
    for sth in range(10)
]

In [11]:
%%px --local
def from_params_index_to_params_values(index_dict):
    params_dict = {k: v for k, v in index_dict.items()}
    for k, v in index_dict.items():
        if k in ['D', 'pgcost', 'seed']: params_dict[k] = params_index_to_value[k][v]
    return params_dict

Async parallel execution on engine(s): all


<AsyncResult: finished>

In [12]:
%%px --local
snapshots_shape = (251, 300, 300)
frequencies_shape = (251, 8)

Async parallel execution on engine(s): all


<AsyncResult: finished>

    for params in make_params_list():
        params_values = from_params_index_to_params_values(params)
        seed = params_values['seed']
        pgcost = params_values['pgcost']
        D = params_values['D']
        sth = params_values['sth']
        cth = params_values['cth']
        !mkdir -p hdf5-data/seed-$seed/pgcost-$pgcost/D-$D/sth-$sth/cth-$cth/

In [13]:
%%px --local
hdf5_filename_templ = 'hdf5-data/seed-{seed}/pgcost-{pgcost}/D-{D}/sth-{sth}/cth-{cth}/{dataset_name}.h5'

Async parallel execution on engine(s): all


<AsyncResult: finished>

```python
def initiate_hdf5_file(params, dataset_name, shape, dtype='uint32'):
    params_values = from_params_index_to_params_values(params)
    hdf5_filename = hdf5_filename_templ.format(dataset_name=dataset_name, **params_values)
    with h5py.File(hdf5_filename) as ff:
        dataset = ff.create_dataset(
            dataset_name,
            shape=shape,
            compression='lzf',
            dtype=dtype,
            fletcher32=True,
        )
        dataset.dims[0].label = 'generation_index'
```

```python
for params in make_params_list():
    initiate_hdf5_file(
        params,
        dataset_name=sample_snapshots,
        shape=snapshots_shape,
        dtype='uint8',
    )
    initiate_hdf5_file(
        params,
        dataset_name=sample_frequencies,
        shape=frequencies_shape,
        dtype='uint32',
    )
```

In [38]:
%%px
def save_datasets(data_array_dict, **params_values):
    with h5py.File(hdf5_filename_templ.format(dataset_name=sample_frequencies, **params_values)) as ff:
        ff[sample_frequencies][:] = data_array_dict[sample_frequencies]
    with h5py.File(hdf5_filename_templ.format(dataset_name=sample_snapshots, **params_values)) as ff:
        ff[sample_snapshots][:] = data_array_dict[sample_snapshots]

Async parallel execution on engine(s): all


<AsyncResult: execute>

In [39]:
%%px --local
def from_json_to_hdf5(params_list):
    for params in params_list:
        params_values = from_params_index_to_params_values(params)
        should_convert = True
        with h5py.File(hdf5_filename_templ.format(dataset_name=sample_frequencies, **params_values)) as ff:
            if (ff[sample_frequencies][...] > 0).all():
                should_convert = False
        with h5py.File(hdf5_filename_templ.format(dataset_name=sample_snapshots, **params_values)) as ff_snapshots:
            if (ff_snapshots[sample_snapshots][...] > 0).all():
                should_convert = False
        if should_convert:
            data_array_dict = strife.get_data_core(**params_values)
            save_datasets(data_array_dict=data_array_dict, **params_values)

Async parallel execution on engine(s): all


<AsyncResult: execute>

In [40]:
dv.scatter('params_list', make_params_index_dict_list())

<AsyncResult: scatter>

In [41]:
%%px -o conversion_result
from_json_to_hdf5(params_list)

Async parallel execution on engine(s): all


<AsyncResult: execute>

In [51]:
conversion_result.elapsed/60/60

27.68613409416667